In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Faça o melhor pre-processamento possível para tornar tal dataset adequado para uso em diversos algoritmos de aprendizagem de máquina.

In [2]:
data = pd.read_csv('datasets/adult.data.csv', header=None)
data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'class']

data = data.drop('education', axis=1)

In [3]:
def cat2numeric_map(data):
    map_replace = {}
    for column in data:
        if not np.issubdtype(data[column].dtype, np.number):
            elements = np.unique(data[column])
            map_replace[column] = { elements[i]: i for i in range(elements.size) }  
    return map_replace

In [4]:
map_replace = cat2numeric_map(data)
map_replace

{'class': {' <=50K': 0, ' >50K': 1},
 'marital-status': {' Divorced': 0,
  ' Married-AF-spouse': 1,
  ' Married-civ-spouse': 2,
  ' Married-spouse-absent': 3,
  ' Never-married': 4,
  ' Separated': 5,
  ' Widowed': 6},
 'native-country': {' ?': 0,
  ' Cambodia': 1,
  ' Canada': 2,
  ' China': 3,
  ' Columbia': 4,
  ' Cuba': 5,
  ' Dominican-Republic': 6,
  ' Ecuador': 7,
  ' El-Salvador': 8,
  ' England': 9,
  ' France': 10,
  ' Germany': 11,
  ' Greece': 12,
  ' Guatemala': 13,
  ' Haiti': 14,
  ' Holand-Netherlands': 15,
  ' Honduras': 16,
  ' Hong': 17,
  ' Hungary': 18,
  ' India': 19,
  ' Iran': 20,
  ' Ireland': 21,
  ' Italy': 22,
  ' Jamaica': 23,
  ' Japan': 24,
  ' Laos': 25,
  ' Mexico': 26,
  ' Nicaragua': 27,
  ' Outlying-US(Guam-USVI-etc)': 28,
  ' Peru': 29,
  ' Philippines': 30,
  ' Poland': 31,
  ' Portugal': 32,
  ' Puerto-Rico': 33,
  ' Scotland': 34,
  ' South': 35,
  ' Taiwan': 36,
  ' Thailand': 37,
  ' Trinadad&Tobago': 38,
  ' United-States': 39,
  ' Vietnam': 4

In [5]:
map_replace = {'workclass': {' ?': np.nan},
 'occupation': {' ?': np.nan},
 'native-country': {' ?': np.nan}}

data = data.replace(map_replace)
data.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
class                0
dtype: int64

In [6]:
data = data.dropna(axis=0)
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

## Faça atribuição da média da coluna para valores faltantes, caso o percentual de valores faltantes da coluna não seja muito grande.

## Transforme categorias usadas nas features e label em números. Categorias que não possuem uma ordem implícita devem ser transformadas em features binárias.

In [7]:
map_replace2 = {'class': {' <=50K': 0, ' >50K': 1}, 'sex': {' Female': 0, ' Male': 1}}

data = data.replace(map_replace2)
# data

In [8]:
#race_mapping = {label: idx for idx, label in enumerate(np.unique(data['race']))}
#data['race'] = data['race'].map(race_mapping)

#relationship_mapping = {label: idx for idx, label in enumerate(np.unique(data['relationship']))}
#data['relationship'] = data['relationship'].map(relationship_mapping)

#maritalStatus_mapping = {label: idx for idx, label in enumerate(np.unique(data['marital-status']))}
#data['marital-status'] = data['marital-status'].map(maritalStatus_mapping)

#workclass_mapping = {label: idx for idx, label in enumerate(np.unique(data['workclass']))}
#data['workclass'] = data['workclass'].map(workclass_mapping)

#occupation_mapping = {label: idx for idx, label in enumerate(np.unique(data['occupation']))}
#data['occupation'] = data['occupation'].map(occupation_mapping)

#nativeCountry_mapping = {label: idx for idx, label in enumerate(np.unique(data['native-country']))}
#data['native-country'] = data['native-country'].map(nativeCountry_mapping)

#data

In [9]:
data2 = pd.get_dummies(data[['age', 'workclass', 'fnlwgt', 'education-num', 'marital-status', 'occupation', 
'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']])

#data2.shape
#data2.columns
data2

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,class,workclass_ Federal-gov,workclass_ Local-gov,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,1,2174,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,1,0,0,13,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,1,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,37,284582,14,0,0,0,40,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,49,160187,5,0,0,0,16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,52,209642,9,1,0,0,45,1,0,0,...,0,0,0,0,0,0,0,1,0,0
8,31,45781,14,0,14084,0,50,1,0,0,...,0,0,0,0,0,0,0,1,0,0
9,42,159449,13,1,5178,0,40,1,0,0,...,0,0,0,0,0,0,0,1,0,0


# Use 5-Fold Cross Validation Estratificada para obter a Accuracy através do uso dos seguintes algoritmos: Logistic Regression, kNN, Naive Bayes, SVM.

## Faça Standardization dos dados (fit para o conjunto de treino e transform para treino e teste).

In [10]:
X = np.hstack((data2.iloc[:,:7].values, data2.iloc[:,8:87].values))
y = data2.iloc[:,7].values

aux = StratifiedKFold(n_splits = 5, shuffle = True)
aux.get_n_splits();

In [11]:
acc_lr = []
acc_knn = []
acc_nb = []
acc_svm = []

for index_train, index_test in aux.split(X, y):
    X_train, X_test = X[index_train], X[index_test]
    y_train, y_test = y[index_train], y[index_test]
    
    std = preprocessing.StandardScaler().fit(X_train)
    X_train_std = std.transform(X_train)
    X_test_std = std.transform(X_test)
    
    #Logistic Regression
    model_lr = LogisticRegression().fit(X_train_std, y_train)
    y_pred_lr = model_lr.predict(X_test_std)
    acc_lr.append(metrics.accuracy_score(y_test, y_pred_lr))
    
    #KNN
    model_knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train_std, y_train)
    y_pred_knn = model_knn.predict(X_test_std)
    acc_knn.append(metrics.accuracy_score(y_test, y_pred_knn))
    
    #Naive Bayes
    model_nb = GaussianNB().fit(X_train_std, y_train)
    y_pred_nb = model_nb.predict(X_test_std)
    acc_nb.append(metrics.accuracy_score(y_test, y_pred_nb))
    
    #SVM
    model_svm = SVC().fit(X_train_std, y_train)
    y_pred_svm = model_svm.predict(X_test_std)
    acc_svm.append(metrics.accuracy_score(y_test, y_pred_svm))
    
    
print(" Accuracy Logistic Regression: ", np.mean(acc_lr))
print("                 Accuracy KNN: ", np.mean(acc_knn))
print("         Accuracy Naive Bayes: ", np.mean(acc_nb))
print("                 Accuracy SVM: ", np.mean(acc_svm))

/home/rodmds/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


 Accuracy Logistic Regression:  0.847324527766
                 Accuracy KNN:  0.816159661001
         Accuracy Naive Bayes:  0.506031954474
                 Accuracy SVM:  0.843909628222


# Escolha a melhor técnica e crie um modelo final usando todo o dataset para treinar o modelo.

In [12]:
model_final = LogisticRegression().fit(X, y)
model_final

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)